<a href="https://colab.research.google.com/github/EwelinaS1/NBP/blob/main/Kopia_zadanie_NBP_9_06_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from datetime import datetime, timedelta
from IPython.display import display, HTML

# Pobieranie danych z NBP
def fetch_currency_data(currency_code, start_date, end_date):
    all_data = []
    current_start = start_date
    max_days = 93

    while current_start <= end_date:
        current_end = min(current_start + timedelta(days=max_days-1), end_date)
        url = (f"http://api.nbp.pl/api/exchangerates/rates/A/{currency_code}/"
               f"{current_start.strftime('%Y-%m-%d')}/{current_end.strftime('%Y-%m-%d')}/?format=json")
        try:
            r = requests.get(url)
            if r.status_code == 200:
                data = r.json()
                all_data.extend(data['rates'])
                print(f"Pobrano dane dla {currency_code} od {current_start.date()} do {current_end.date()}")
            else:
                print(f"Błąd pobierania danych dla {currency_code}: HTTP {r.status_code}")
                break
        except Exception as e:
            print(f"Wyjątek podczas pobierania danych dla {currency_code}: {e}")
            break
        current_start = current_end + timedelta(days=1)

    df = pd.DataFrame(all_data)
    if not df.empty:
        df['currency'] = currency_code
    return df

# Daty i waluty
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
currencies = ['EUR', 'USD', 'GBP', 'CHF']

# Pobieranie danych
dfs = []
for cur in currencies:
    df = fetch_currency_data(cur, start_date, end_date)
    if not df.empty:
        dfs.append(df)

if not dfs:
    raise ValueError("Nie pobrano żadnych danych!")

# Przetwarzanie danych
all_data = pd.concat(dfs)
all_data['effectiveDate'] = pd.to_datetime(all_data['effectiveDate'])
pivot_df = all_data.pivot(index='effectiveDate', columns='currency', values='mid').sort_index()
pct_change = pivot_df.pct_change() * 100
rolling_mean = pivot_df.rolling(window=30).mean()
corr = pivot_df.corr()

# Wydarzenia geopolityczne
events = {
    "2023-03-10": "Upadek Silicon Valley Bank (SVB)",
    "2023-07-27": "ECB podnosi stopy procentowe o 25 bp",
    "2023-10-07": "Atak Hamasu na Izrael (Operation Al‑Aqsa Flood)",
    "2024-03-05": "Super Tuesday – prawybory USA",
    "2024-06-12": "ECB obniża stopy procentowe o 25 bp",
    "2024-11-05": "Wybory prezydenckie w USA",
}

events_df = pd.DataFrame({
    'date': pd.to_datetime(list(events.keys())),
    'event': list(events.values())
})

# Wskazówka dla użytkownika
event_list_html = "<ul>" + "".join([f"<li><b>{date}</b>: {desc}</li>" for date, desc in events.items()]) + "</ul>"

display(HTML(
    f"""
    <div style='margin-bottom: 20px;'>
        <b>Wskazówka:</b><br>
        • Najedź kursorem na <span style="color: red;">czerwoną linię</span>, aby zobaczyć opis wydarzenia.<br>
        • Kliknij w legendę, aby ukrywać/pokazywać konkretne waluty.<br>
        • Użyj suwaka poniżej wykresu, aby wybrać interesujący zakres dat.<br><br>
        <b>Wydarzenia geopolityczne uwzględnione na wykresach:</b>
        {event_list_html}
    </div>
    """
))

# Tworzenie wykresów
def create_figure(df, title, yaxis_title):
    fig = go.Figure()

    # Linie walut
    for cur in currencies:
        fig.add_trace(go.Scatter(x=df.index, y=df[cur], mode='lines', name=cur))

    # Linie i punkty
    for _, row in events_df.iterrows():
        # Marker
        fig.add_trace(go.Scatter(
            x=[row['date']],
            y=[df.max().max()],
            mode='markers',
            marker=dict(color='red', size=8, symbol='line-ns-open'),
            name='Wydarzenie',
            hovertemplate=f"<b>{row['event']}</b><br>{row['date'].date()}<extra></extra>",
            showlegend=False
        ))
        # Linia pionowa
        fig.add_vline(x=row['date'], line=dict(color='red', width=1, dash='dot'))

    fig.update_layout(
        title=title,
        xaxis_title="Data",
        yaxis_title=yaxis_title,
        hovermode='x unified',
        margin=dict(l=40, r=20, t=80, b=80),
        xaxis=dict(rangeslider=dict(visible=True), type="date")
    )
    return fig

# Tytuły wykresów
fig1 = create_figure(pivot_df, "Kursy walut a wydarzenia geopolityczne na świecie", "Kurs waluty")
fig2 = create_figure(pct_change, "Procentowe zmiany kursów walut", "Zmiana [%]")
fig3 = create_figure(rolling_mean, "Średnia krocząca kursów walut (okres 30 sesji)", "Kurs waluty")

# Macierz
fig4 = px.imshow(corr, text_auto=True, color_continuous_scale='RdBu_r', origin='lower',
                 title="Macierz korelacji kursów walut 2023–2024")
fig4.update_layout(margin=dict(l=40, r=20, t=80, b=50))

# Wyświetlanie wykresów
fig1.show()
fig2.show()
fig3.show()
fig4.show()

# Eksport wykresów do plików HTML z interaktywnymi wykresami
fig1.write_html("kursy_walut_wydarzenia.html")
fig2.write_html("procentowe_zmiany_kursow.html")
fig3.write_html("srednia_kroczaca_kursow.html")
fig4.write_html("macierz_korelacji_kursow.html")

print("Wykresy zostały zapisane jako pliki HTML w katalogu roboczym.")


Pobrano dane dla EUR od 2023-01-01 do 2023-04-03
Pobrano dane dla EUR od 2023-04-04 do 2023-07-05
Pobrano dane dla EUR od 2023-07-06 do 2023-10-06
Pobrano dane dla EUR od 2023-10-07 do 2024-01-07
Pobrano dane dla EUR od 2024-01-08 do 2024-04-09
Pobrano dane dla EUR od 2024-04-10 do 2024-07-11
Pobrano dane dla EUR od 2024-07-12 do 2024-10-12
Pobrano dane dla EUR od 2024-10-13 do 2024-12-31
Pobrano dane dla USD od 2023-01-01 do 2023-04-03
Pobrano dane dla USD od 2023-04-04 do 2023-07-05
Pobrano dane dla USD od 2023-07-06 do 2023-10-06
Pobrano dane dla USD od 2023-10-07 do 2024-01-07
Pobrano dane dla USD od 2024-01-08 do 2024-04-09
Pobrano dane dla USD od 2024-04-10 do 2024-07-11
Pobrano dane dla USD od 2024-07-12 do 2024-10-12
Pobrano dane dla USD od 2024-10-13 do 2024-12-31
Pobrano dane dla GBP od 2023-01-01 do 2023-04-03
Pobrano dane dla GBP od 2023-04-04 do 2023-07-05
Pobrano dane dla GBP od 2023-07-06 do 2023-10-06
Pobrano dane dla GBP od 2023-10-07 do 2024-01-07
Pobrano dane dla GBP

Wykresy zostały zapisane jako pliki HTML w katalogu roboczym.
